In [1]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from torch import nn, optim

In [2]:
data = r'D:\장우영\LOCALSEARCH\DA\DA\data\FAmerge_20230523_161834.csv'
merge_data2 = pd.read_csv(data, encoding='ANSI')
print(merge_data2.head(1))

        mmsi ship_name  ship_type   
0  440051540      D-01          0  \

                                                geom    cog  sog   
0  0101000020110F00000000002039676B41000000002DCF...  329.2  5.7  \

        insert_time  지방청           표지                일시   풍향   유향    기온    수온   
0  2023-05-11 10:10  부산청  신항유도등부표(랜비)  2023-05-11 10:10  196  246  16.4  17.0  \

     풍속   유속    기압  습도  
0  9.77  0.3  1017  82  


In [3]:
from pyproj import Transformer

# pyproj Transformer 객체 생성 (3857 좌표계 -> 4326 좌표계)
transformer = Transformer.from_crs(3857, 4326, always_xy=True)

def convert_coordinates(coord):
    # 빈 문자열인 경우 처리
    if not coord:
        return None

    try:
        # 좌표값을 파싱하여 x, y 좌표 추출
        x = int(coord[8:24], 16) / 10000000.0
        y = int(coord[24:40], 16) / 10000000.0

        # 좌표 변환
        lon, lat = transformer.transform(x, y)

        return lon, lat
    except ValueError:
        # 유효하지 않은 형식의 좌표값인 경우 처리
        return None

# 좌표 변환 적용 및 데이터프레임 업데이트
merge_data2['geom'] = merge_data2['geom'].apply(convert_coordinates)

# 변환된 좌표값 출력 또는 원하는 작업 수행

# 데이터프레임에서 '경도'와 '위도' 열로 분리하여 저장
merge_data2['경도'] = merge_data2['geom'].apply(lambda x: x[0] if x is not None else None)
merge_data2['위도'] = merge_data2['geom'].apply(lambda x: x[1] if x is not None else None)

In [4]:
print(merge_data2["geom"])

0        (-72.66076918508794, 90.0)
1        (-72.66076918508794, 90.0)
2        (-72.66076918508794, 90.0)
3        (-72.66076918508794, 90.0)
4        (-72.66076918508794, 90.0)
                    ...            
46809    (-72.66076918508794, 90.0)
46810    (-72.66076918508794, 90.0)
46811    (-72.66076918508794, 90.0)
46812    (-72.66076918508794, 90.0)
46813    (-72.66076918508794, 90.0)
Name: geom, Length: 46814, dtype: object


In [5]:
# 'insert_time' 열을 datetime 개체로 변환한 다음 Unix 타임스탬프로 변환하여 숫자 표현으로 변환
merge_data2['insert_time'] = pd.to_datetime(merge_data2['insert_time']).astype('int64') // 10**9

In [6]:
print(merge_data2["insert_time"])

0        1683799800
1        1683799800
2        1683799800
3        1683799800
4        1683799800
            ...    
46809    1683803340
46810    1683803340
46811    1683803340
46812    1683803340
46813    1683803400
Name: insert_time, Length: 46814, dtype: int64


In [7]:
# 데이터를 교육 및 테스트 세트로 분할
X = merge_data2[["mmsi","ship_name","ship_type","cog","sog","insert_time","풍향","기온","수온","풍속","유속","기압","습도"]]  # 입력 값 (excluding "geom") # 독립변수
y = merge_data2["geom"]  # 출력 값 (geom)       # 종속변수 

print(X)
print(y)

            mmsi             ship_name  ship_type    cog  sog  insert_time   
0      440051540                  D-01          0  329.2  5.7   1683799800  \
1      440300780                   NaN          0  329.8  0.0   1683799800   
2      440208550                   NaN          0  139.3  0.0   1683799800   
3      440275000         GBK EXPRESS 1         40  175.8  0.0   1683799800   
4      440414850          NO1 GEO SUNG         80    0.0  0.0   1683799800   
...          ...                   ...        ...    ...  ...          ...   
46809  440233000  PANSTAR TSUSHIMA LIN         60  360.0  0.1   1683803340   
46810  440108720             KYEUNG UN         52  176.4  0.3   1683803340   
46811  440133280                   NaN          0  331.3  5.3   1683803340   
46812  440132370             KOMAR NO2         80  183.2  7.6   1683803340   
46813  440800234                   NaN          0  217.0  0.0   1683803400   

        풍향    기온    수온     풍속   유속    기압  습도  
0      196  16.4

In [8]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=

0.2, random_state=42)

#학습용, 검증용, 시험용으로 분리
train_size = int(327 * 0.8)
X_train, y_train = X[:train_size], y[:train_size]
X_val, y_val = X[train_size:train_size+33], y[train_size:train_size+33]
X_test, y_test = X[train_size+33:], y[train_size+33:]

In [11]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from sklearn.preprocessing import OneHotEncoder

#  "ship_name","ship_type" 의 형식이 너무 번잡하여 변수를 원-핫 인코딩하여 데이터프레임에 추가
categorical_cols = ["ship_name", "ship_type"]
encoder = OneHotEncoder(sparse=False)
X_train_encoded = pd.DataFrame(encoder.fit_transform(merge_data2[categorical_cols]))
X_train = pd.concat([merge_data2.drop(categorical_cols, axis=1), X_train_encoded], axis=1)

# 입력 기능( X_train)을 numpy 배열로 변환합니다.
X_train = X_train.values

#  y_train 튜플을 별도의 열 형식로 변환
#y_train = np.array([coord if coord is not None else (0, 0) for coord in y_train], dtype=np.float32)
#y_train = np.array([str(coord) if coord is not None else '0,0' for coord in y_train])  
#y_train = np.array([f"{coord[0]},{coord[1]}" if coord is not None else '0,0' for coord in y_train])
y_train = np.array([f"{coord[0]},{coord[1]}" if isinstance(coord, tuple) else '0,0' for coord in y_train])


# Convert X_train to float32 data type  
X_train = X_train.astype(np.float32)
y_train = np.array(y_train, dtype=np.float32)
#y_train = y_train.astype(np.float32)

# Build the model
model = Sequential()
model.add(LSTM(128, input_shape=(X_train.shape[1], X_train.shape[2])))
model.add(Dense(2))
model.compile(loss='mean_squared_error', optimizer='adam')

# Convert X_train and y_train to TensorFlow Tensors
X_train = tf.convert_to_tensor(X_train)
y_train = tf.convert_to_tensor(y_train)

#  tf.convert_to_tensor(). 를 사용하여 100 epochs 및 32의 배치 크기에 대한 학습 데이터( X_train및 )를 사용하여 모델을 학습
model.fit(X_train, y_train, epochs=100, batch_size=32)


c:\Users\SW\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\preprocessing\_encoders.py:868: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


Unexpected exception formatting exception. Falling back to standard exception


TypeError: float() argument must be a string or a real number, not 'tuple'

The above exception was the direct cause of the following exception:

Traceback (most recent call last):
  File "C:\Users\SW\AppData\Roaming\Python\Python310\site-packages\IPython\core\interactiveshell.py", line 3505, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "C:\Users\SW\AppData\Local\Temp\ipykernel_12888\4132456713.py", line 25, in <module>
    X_train = X_train.astype(np.float32)
ValueError: setting an array element with a sequence.

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "C:\Users\SW\AppData\Roaming\Python\Python310\site-packages\IPython\core\interactiveshell.py", line 2102, in showtraceback
    stb = self.InteractiveTB.structured_traceback(
  File "C:\Users\SW\AppData\Roaming\Python\Python310\site-packages\IPython\core\ultratb.py", line 1310, in structured_traceback
    return FormattedTB.structured_trace

In [ ]:
# y_train데이터에 형식의 튜플이 포함된 경우 (-72.66076918508794, 90.0)작업에 좌표 쌍(경도, 위도) 예측이 포함되는 것 같습니다. 
# 이 경우 LSTM 모델 교육에 대한 접근 방식을 수정해야 합니다.

# 예측 작업을 단일 출력 노드가 있는 회귀 문제로 취급하는 대신 각 출력이 좌표 쌍인 시퀀스 간 문제로 재구성해야 합니다.

# 다음은 코드를 수정하기 위해 따를 수 있는 접근 방식입니다.

# 대상 변수 수정( y_train): 튜플을 y_train경도와 위도를 나타내는 별도의 열로 변환합니다. y_train_longitude경도 값을 포함하고
# y_train_latitude위도 값을 포함하는 두 개의 개별 배열을 만듭니다 .

# 모델 아키텍처 수정: 마지막 Dense레이어에 단일 출력 노드를 두는 대신 두 개의 출력 노드가 있어야 합니다. 
# 하나는 경도용이고 다른 하나는 위도용입니다.

# 손실 함수 조정: 이제 두 개의 개별 값을 예측하고 있으므로 이에 따라 손실 함수를 수정해야 합니다.
# 각 출력에 대해 평균 제곱 오차(MSE)와 같이 경도와 위도에 대해 별도의 손실 함수를 사용할 수 있습니다.